# Convolution demos

In this notebook, we use the batch, multi-channel convolution operation implemented in Numpy (that you can find [here](../lincoln/lincoln/conv.py)) to train a small convolutional neural network to more than 90% accuracy on MNIST.

In [1]:
import numpy as np

# import lincoln
# from lincoln.layers import Dense
# from lincoln.losses import SoftmaxCrossEntropy, MeanSquaredError
# from lincoln.optimizers import Optimizer, SGD, SGDMomentum
# from lincoln.activations import Sigmoid, Tanh, Linear, ReLU
# from lincoln.network import NeuralNetwork
# from lincoln.train import Trainer
# from lincoln.utils import mnist
# from lincoln.layers import Conv2D

from dlfs_kuoste import helpers
from dlfs_kuoste import layers
from dlfs_kuoste import losses
from dlfs_kuoste import optimizers
from dlfs_kuoste import operations
from dlfs_kuoste import NeuralNetwork
from dlfs_kuoste import Trainer

X_train, y_train, X_test, y_test = helpers.mnist_load()

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
X_train, X_test = X_train - np.mean(X_train), X_test - np.mean(X_train)
X_train, X_test = X_train / np.std(X_train), X_test / np.std(X_train)

In [4]:
X_train_conv, X_test_conv = X_train.reshape(-1, 1, 28, 28), X_test.reshape(-1, 1, 28, 28)

In [5]:
num_labels = len(y_train)
train_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    train_labels[i][y_train[i]] = 1

num_labels = len(y_test)
test_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    test_labels[i][y_test[i]] = 1

In [6]:
def calc_accuracy_model(model, test_set):
    return print(f'''The model validation accuracy is: 
    {np.equal(np.argmax(model.forward(test_set, inference=True), axis=1), y_test).sum() * 100.0 / test_set.shape[0]:.2f}%''')

# CNN from scratch

In [7]:
model = NeuralNetwork(
    layers=[layers.Conv2D(out_channels=16,
                   param_size=5,
                   dropout=0.8,
                   weight_init="glorot",
                   flatten=True,
                  activation=operations.Tanh()),
            layers.Dense(neurons=10, 
                  activation=operations.Linear())],
            loss = losses.SoftmaxCrossEntropy(), 
seed=20190402)

trainer = Trainer(model, optimizers.SgdMomentum(lr = 0.1, momentum=0.9))
trainer.fit(X_train_conv, train_labels, X_test_conv, test_labels,
            epochs = 1,
            eval_every = 1,
            seed=20190402,
            batch_size=60,
            conv_testing=True);

batch 0 loss 31.191501893742405
batch 10 loss 14.150390490525517
batch 20 loss 8.507022907684597
batch 30 loss 9.81608465956318
batch 40 loss 2.7069227167896712
batch 50 loss 5.039126892843199
batch 60 loss 3.841335350417124
batch 70 loss 8.47307919757578
batch 80 loss 5.373412749700996
batch 90 loss 2.42436564041577
batch 100 loss 4.124895016673244
Validation accuracy after 100 batches is 85.56%
batch 110 loss 9.333326175947795
batch 120 loss 5.668763088393043
batch 130 loss 8.030682647094096
batch 140 loss 1.5587190577707137
batch 150 loss 6.860395904028616
batch 160 loss 4.220174743632938
batch 170 loss 5.190430233727453
batch 180 loss 4.674891852621207
batch 190 loss 5.2314336031322535
batch 200 loss 5.062087142757434
Validation accuracy after 200 batches is 85.74%
batch 210 loss 3.311997869393835
batch 220 loss 5.023688325618068
batch 230 loss 4.563027966764308
batch 240 loss 3.344317953584475
batch 250 loss 3.5711253470995854
batch 260 loss 5.177948164842362
batch 270 loss 6.2170

In [8]:
calc_accuracy_model(model, X_test_conv)

The model validation accuracy is: 
    91.47%
